# nearest gene

In [1]:
import yaml
from pathlib import Path
import pandas as pd
import os



CONFIG = Path("/data/ruderferlab/projects/ctcf/projects/cbs-db3/cbs-db3-annotate/config/config.yaml")
ENCODE_NEAREST = "GRCh38-Closest-Genes-PC.gz"
# GENCODE_MAPPING = "gencode.v47.basic.annotation.parsed.pc.longest_tx.tsv"


def parse_config(config_path: str) -> dict:
    "Parses provided YAML configuration file as a dictionary."
    with open(config_path, "r", encoding="utf-8") as file:
        config = yaml.safe_load(file)
    return config

def parse_nearest_gene(filepath: str) -> pd.DataFrame:
     """d"""
     return pd.read_csv(
        filepath,
        sep="\t",
        header=None,
        usecols=[4, 9, 12],
        names=["ccre_accession", "tx_id", "gene_id"],
        dtype={"ccre_accession": str, "tx_id": str, "gene_id": str},
        engine="c",
        compression="gzip",
        )

# def parse_gencode(filepath: str) -> pd.DataFrame:
#     """d"""
#     return pd.read_csv(
#         filepath,
#         sep="\t",
#         header=None,
#         names=["gene_name", "gene_id", "tx_id"],
#         dtype={"gene_name": str, "gene_id": str, "tx_id": str},
#         engine="c",
#         )
    
###
# Main
###

# Parse config
try:
    config_file = parse_config(CONFIG.resolve())
except FileNotFoundError:
    print("Error: config not found.")
    
# Register inputs from resource  directory
ENCODE_DIR = Path(config_file["RESOURCES"]["ENCODE"]["SCREEN"]["V4"])
# GENCODE_DIR = Path(config_file["RESOURCES"]["GENCODE"]["V47"])

# Register input files
encode_nearest_gene = ENCODE_DIR / ENCODE_NEAREST
# gencode_mapping = "/data/ruderferlab/projects/ctcf/projects/cbs-db3/cbs-db3-annotate/resources/data/gencode/v47/gencode.v47.basic.annotation.parsed.pc.longest_tx.tsv"

# Parse input files
nearest_gene = parse_nearest_gene(encode_nearest_gene)
# gencode_mapping = parse_gencode(gencode_mapping)

In [2]:
nearest_gene

,ccre_accession,tx_id,gene_id
0,EH38E2776516,ENST00000641515.2,ENSG00000186092.7
1,EH38E2776517,ENST00000641515.2,ENSG00000186092.7
2,EH38E3951272,ENST00000641515.2,ENSG00000186092.7
3,EH38E3951273,ENST00000641515.2,ENSG00000186092.7
4,EH38E3951274,ENST00000641515.2,ENSG00000186092.7
...,...,...,...
2651782,EH38E3951270,ENST00000361963.3,ENSG00000172288.7
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7
2651784,EH38E4535242,ENST00000361963.3,ENSG00000172288.7
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y


In [ ]:
###
# For now get 1:1 mapping by dropping duplicates (development)
###

nearest_gene = nearest_gene.drop_duplicates(subset=["ccre_accession"], keep="first")

# Trim trailing "." from gene_id and tx_id
nearest_gene.loc[:, "tx_id"] = nearest_gene["tx_id"].str.split(".").str[0]
nearest_gene.loc[:, "gene_id"] = nearest_gene["gene_id"].str.split(".").str[0]

# Write out
OUTDIR = "/data/ruderferlab/projects/ctcf/projects/cbs-db3/cbs-db3-annotate/results/annotations/encode/screen/v4"
OUT_FILE = "closest_pc_gene.tsv.anno.gz"
nearest_gene.to_csv(
    os.path.join(OUTDIR, OUT_FILE),
    sep="\t",
    index=False,
    header=True,
    compression="gzip",
)

In [9]:
nearest_gene

,ccre_accession,tx_id,gene_id
0,EH38E2776516,ENST00000641515,ENSG00000186092
1,EH38E2776517,ENST00000641515,ENSG00000186092
2,EH38E3951272,ENST00000641515,ENSG00000186092
3,EH38E3951273,ENST00000641515,ENSG00000186092
4,EH38E3951274,ENST00000641515,ENSG00000186092
...,...,...,...
2651779,EH38E3951269,ENST00000306609,ENSG00000172288
2651781,EH38E3951270,ENST00000306609,ENSG00000172288
2651783,EH38E4535242,ENST00000306609,ENSG00000172288
2651785,EH38E4535243,ENST00000286448,ENSG00000124333


In [27]:
gencode_mapping

,gene_name,gene_id,tx_id
0,OR4F5,ENSG00000186092.7,ENST00000641515.2
1,OR4F29,ENSG00000284733.2,ENST00000426406.4
2,OR4F16,ENSG00000284662.2,ENST00000332831.5
3,SAMD11,ENSG00000187634.13,ENST00000616016.5
4,NOC2L,ENSG00000188976.11,ENST00000327044.7
...,...,...,...
20087,MT-ND4L,ENSG00000212907.2,ENST00000361335.1
20088,MT-ND4,ENSG00000198886.2,ENST00000361381.2
20089,MT-ND5,ENSG00000198786.2,ENST00000361567.2
20090,MT-ND6,ENSG00000198695.2,ENST00000361681.2


In [24]:
nearest_gene.drop_duplicates()['ccre_accession'].value_counts()

ccre_accession
EH38E3297036    4
EH38E1951203    4
EH38E2571346    4
EH38E3024082    3
EH38E2761830    3
               ..
EH38E3172994    1
EH38E3172993    1
EH38E3172992    1
EH38E1807374    1
EH38E4535244    1
Name: count, Length: 2348854, dtype: int64

In [28]:
nearest_gene[nearest_gene['ccre_accession']=="EH38E3297036"]

,ccre_accession,tx_id,gene_id
1144859,EH38E3297036,ENST00000444486.7,ENSG00000213999.17
1144860,EH38E3297036,ENST00000514819.7,ENSG00000064489.23
1144861,EH38E3297036,ENST00000585679.1,ENSG00000254901.8
1144862,EH38E3297036,ENST00000462790.8,ENSG00000254901.8
1144863,EH38E3297036,ENST00000456252.7,ENSG00000064490.14
1144864,EH38E3297036,ENST00000303088.9,ENSG00000064490.14


In [10]:
nearest_gene.drop_duplicates(subset=["ccre_accession"])

,ccre_accession,tx_id,gene_id
0,EH38E2776516,ENST00000641515.2,ENSG00000186092.7
1,EH38E2776517,ENST00000641515.2,ENSG00000186092.7
2,EH38E3951272,ENST00000641515.2,ENSG00000186092.7
3,EH38E3951273,ENST00000641515.2,ENSG00000186092.7
4,EH38E3951274,ENST00000641515.2,ENSG00000186092.7
...,...,...,...
2651779,EH38E3951269,ENST00000306609.4,ENSG00000172288.7
2651781,EH38E3951270,ENST00000306609.4,ENSG00000172288.7
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y


In [16]:
nearest_gene

,ccre_accession,tx_id,gene_id
0,EH38E2776516,ENST00000641515.2,ENSG00000186092.7
1,EH38E2776517,ENST00000641515.2,ENSG00000186092.7
2,EH38E3951272,ENST00000641515.2,ENSG00000186092.7
3,EH38E3951273,ENST00000641515.2,ENSG00000186092.7
4,EH38E3951274,ENST00000641515.2,ENSG00000186092.7
...,...,...,...
2651782,EH38E3951270,ENST00000361963.3,ENSG00000172288.7
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7
2651784,EH38E4535242,ENST00000361963.3,ENSG00000172288.7
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y


In [19]:
x = nearest_gene.merge(
    gencode_mapping,
    how="left",
    on=['gene_id'],
)



In [20]:
x

,ccre_accession,tx_id_x,gene_id,gene_name,tx_id_y
0,EH38E2776516,ENST00000641515.2,ENSG00000186092.7,OR4F5,ENST00000641515.2
1,EH38E2776517,ENST00000641515.2,ENSG00000186092.7,OR4F5,ENST00000641515.2
2,EH38E3951272,ENST00000641515.2,ENSG00000186092.7,OR4F5,ENST00000641515.2
3,EH38E3951273,ENST00000641515.2,ENSG00000186092.7,OR4F5,ENST00000641515.2
4,EH38E3951274,ENST00000641515.2,ENSG00000186092.7,OR4F5,ENST00000641515.2
...,...,...,...,...,...
2651782,EH38E3951270,ENST00000361963.3,ENSG00000172288.7,NaN,NaN
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7,NaN,NaN
2651784,EH38E4535242,ENST00000361963.3,ENSG00000172288.7,NaN,NaN
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y,NaN,NaN


In [9]:
nearest_gene['ccre_accession'].nunique()

2348854

In [11]:
nearest_gene.head()

,ccre_accession,tx_id,gene_id
0,EH38E2776516,ENST00000641515.2,ENSG00000186092.7
1,EH38E2776517,ENST00000641515.2,ENSG00000186092.7
2,EH38E3951272,ENST00000641515.2,ENSG00000186092.7
3,EH38E3951273,ENST00000641515.2,ENSG00000186092.7
4,EH38E3951274,ENST00000641515.2,ENSG00000186092.7


In [14]:
nearest_gene['ccre_accession'].value_counts()

ccre_accession
EH38E3975884    47
EH38E2320864    45
EH38E2309980    42
EH38E3818239    40
EH38E3539202    37
                ..
EH38E3190762     1
EH38E4148422     1
EH38E3190760     1
EH38E3190759     1
EH38E4535244     1
Name: count, Length: 2348854, dtype: int64

In [15]:
nearest_gene[nearest_gene['ccre_accession'] == 'EH38E3975884'].head()

,ccre_accession,tx_id,gene_id
114671,EH38E3975884,ENST00000685628.1,ENSG00000121940.17
114672,EH38E3975884,ENST00000676454.1,ENSG00000121940.17
114673,EH38E3975884,ENST00000675790.1,ENSG00000121940.17
114674,EH38E3975884,ENST00000674849.1,ENSG00000121940.17
114675,EH38E3975884,ENST00000675956.1,ENSG00000121940.17


In [22]:
nearest_gene[['gene_id','tx_id']].merge(gencode_mapping[['gene_id','tx_id']], on=['gene_id','tx_id'], how='left').dropna()

,gene_id,tx_id
0,ENSG00000186092.7,ENST00000641515.2
1,ENSG00000186092.7,ENST00000641515.2
2,ENSG00000186092.7,ENST00000641515.2
3,ENSG00000186092.7,ENST00000641515.2
4,ENSG00000186092.7,ENST00000641515.2
...,...,...
2651782,ENSG00000172288.7,ENST00000361963.3
2651783,ENSG00000172288.7,ENST00000306609.4
2651784,ENSG00000172288.7,ENST00000361963.3
2651785,ENSG00000124333.16_PAR_Y,ENST00000286448.12_PAR_Y


In [19]:
nearest_gene.merge(
    gencode_mapping,
    how="left",
    on=['gene_id', 'tx_id'])['gene_name'].value_counts(dropna=False)


gene_name
NaN                1606215
TENM3                 1688
TENM4                 1430
SOX9                  1345
UQCRFS1               1337
                    ...   
PSENEN                   1
U2AF1L4                  1
ENSG00000267120          1
IGFLR1                   1
ENSG00000285505          1
Name: count, Length: 17300, dtype: int64

In [5]:
gencode_mapping.head()

,gene_name,gene_id,tx_id
0,OR4F5,ENSG00000186092.7,ENST00000641515.2
1,OR4F29,ENSG00000284733.2,ENST00000426406.4
2,OR4F16,ENSG00000284662.2,ENST00000332831.5
3,SAMD11,ENSG00000187634.13,ENST00000616016.5
4,NOC2L,ENSG00000188976.11,ENST00000327044.7


In [4]:
nearest_gene.sort_values(
    by=["ccre_accession"],
)

,ccre_accession,tx_id,gene_id
72702,EH38E0001314,ENST00000371226.8,ENSG00000162600.12
170113,EH38E0003674,ENST00000367462.5,ENSG00000162670.11
170281,EH38E0003686,ENST00000367462.5,ENSG00000162670.11
197068,EH38E0004320,ENST00000259154.9,ENSG00000136636.13
240234,EH38E0005245,ENST00000298375.12,ENSG00000165568.18
...,...,...,...
2651773,EH38E4535241,ENST00000306609.4,ENSG00000172288.7
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7
2651784,EH38E4535242,ENST00000361963.3,ENSG00000172288.7
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y


In [ ]:
# Register input resource  directory
ip_dir = Path(config_file["FILEPATHS"]["RESOURCE_DIR"]).resolve()

# Register input resource file
ip_file = ip_dir / IP_DATA

# Parse input resource file
nearest_gene = parse_nearest_gene(ip_file)

# Create 1 to 1 mapping of ccre_accession to gene/tx

# Sort
nearest_gene = nearest_gene.sort_values(by=["ccre_accession", "tx_id"])

In [11]:
nearest_gene

,ccre_accession,tx_id,gene_id
72702,EH38E0001314,ENST00000371226.8,ENSG00000162600.12
170113,EH38E0003674,ENST00000367462.5,ENSG00000162670.11
170281,EH38E0003686,ENST00000367462.5,ENSG00000162670.11
197068,EH38E0004320,ENST00000259154.9,ENSG00000136636.13
240234,EH38E0005245,ENST00000298375.12,ENSG00000165568.18
...,...,...,...
2651774,EH38E4535241,ENST00000361963.3,ENSG00000172288.7
2651783,EH38E4535242,ENST00000306609.4,ENSG00000172288.7
2651784,EH38E4535242,ENST00000361963.3,ENSG00000172288.7
2651785,EH38E4535243,ENST00000286448.12_PAR_Y,ENSG00000124333.16_PAR_Y


# 3D chromatin gene target

### 3D Chromatin
| cCRE ID | Gene ID | Common Gene Name | Gene Type | Assay Type | Experiment ID | Biosample | Score | P-value |
|---------|---------|-----------------|-----------|------------|---------------|-----------|--------|----------|
| | | | | | | | | |

In [1]:
import pandas as pd

In [2]:
GENE_TARGETS = "/panfs/accrepfs.vampire/data/ruderferlab/projects/ctcf/resources/data/encode/screen/v4/V4-hg38.Gene-Links.3D-Chromatin.txt.gz"

In [3]:
# Read gene targets
gene_targets = pd.read_csv(
    GENE_TARGETS,
    sep="\t",
    header=None,
    # usecols=[0, 1, 2, 3],
    # names=["ccre_accession", "gene_id", "tx_id", "gene_name"],
    # dtype={"ccre_accession": str, "gene_id": str, "tx_id": str, "gene_name": str},
    engine="c",
    compression="gzip",
)

In [5]:
gene_targets.head()

,0,1,2,3,4,5,6,7,8
0,EH38E0080197,ENSG00000121769,FABP3,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,1.31,NaN
1,EH38E0080197,ENSG00000134644,PUM1,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,10.39,NaN
2,EH38E0082273,ENSG00000171812,COL8A2,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,15.68,NaN
3,EH38E0096620,ENSG00000118454,ANKRD13C,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,1.61,NaN
4,EH38E0096620,ENSG00000197568,HHLA3,lncRNA,RNAPII-ChIAPET,ENCSR782EKZ,H1,1.61,NaN


In [4]:
# Subet to protein coding
gene_targets = gene_targets[gene_targets[3] == "protein_coding"]

In [ ]:
# for each ccre-gene-biosample, select the highest score
highest_scores = gene_targets.groupby(by=[0, 1, 6])[7].max().reset_index()
highest_scores = highest_scores.rename(columns={0: "ccre_accession", 1: "gene_id", 6: "biosample_id", 7: "score"})

,0,1,6,7
0,EH38E0048077,ENSG00000198755,HEK293,6.38
1,EH38E0048077,ENSG00000198755,HEK293T,3.03
2,EH38E0048077,ENSG00000198755,WTC11_genetically_modified_(insertion)_using_C...,2.64
3,EH38E0055676,ENSG00000183779,MCF-7,8.28
4,EH38E0070245,ENSG00000162496,HCT116,9.59
...,...,...,...,...
3813857,EH38E4535008,ENSG00000288049,"naive_thymus-derived_CD4-positive,_alpha-beta_...",14.44
3813858,EH38E4535008,ENSG00000288049,"stimulated_activated_naive_CD4-positive,_alpha...",8.04
3813859,EH38E4535015,ENSG00000131002,OCI-LY7,2.35
3813860,EH38E4535017,ENSG00000012817,WTC11_genetically_modified_(insertion)_using_C...,2.72


In [5]:
gene_targets

,0,1,2,3,4,5,6,7,8
0,EH38E0080197,ENSG00000121769,FABP3,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,1.31,NaN
1,EH38E0080197,ENSG00000134644,PUM1,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,10.39,NaN
2,EH38E0082273,ENSG00000171812,COL8A2,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,15.68,NaN
3,EH38E0096620,ENSG00000118454,ANKRD13C,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,1.61,NaN
5,EH38E0114249,ENSG00000178096,BOLA1,protein_coding,RNAPII-ChIAPET,ENCSR782EKZ,H1,21.65,NaN
...,...,...,...,...,...,...,...,...,...
4168059,EH38E1545247,ENSG00000126391,FRMD8,protein_coding,RNAPII-ChIAPET,ENCSR499JGQ,MCF_10A,62.75,NaN
4168060,EH38E1545247,ENSG00000142186,SCYL1,protein_coding,RNAPII-ChIAPET,ENCSR499JGQ,MCF_10A,3.08,NaN
4168061,EH38E1545247,ENSG00000162241,SLC25A45,protein_coding,RNAPII-ChIAPET,ENCSR499JGQ,MCF_10A,62.75,NaN
4168062,EH38E1545247,ENSG00000173465,ZNRD2,protein_coding,RNAPII-ChIAPET,ENCSR499JGQ,MCF_10A,2.53,NaN


# biosamples

In [ ]:
import pandas as pd

In [1]:
BIOSAMPLE_MAP = "/data/ruderferlab/projects/ctcf/projects/cbs-db3/cbs-db3-annotate/resources/data/encode/screen/v4/biosample_map.tsv"

In [10]:
# Read biosample map
biosample_map = pd.read_csv(
    BIOSAMPLE_MAP,
    sep="\t",
    engine="c",
)

In [11]:
####
# Main
###

# Extract experiment ID
biosample_map["experiment_id"] = biosample_map["value"].str.split("_").str[-1]

# Drop sampleType
biosample_map.drop(columns=["sampleType"], axis=1, inplace=True)

# Set column labels
labels = ["biosample", "tissue", "life_stage", "display_name", "experiment_id"]
biosample_map.columns = labels

# Write out
biosample_map.to_csv("/data/ruderferlab/projects/ctcf/projects/cbs-db3/cbs-db3-annotate/results/tables/encode/screen/v4/biosample_map.tsv.table.gz", sep="\t", index=False, header=True, compression="gzip")

In [8]:
biosample_map

,biosample,tissue,life_stage,display_name,experiment_id
0,22Rv1_ENCDO719HIY,prostate,adult,22Rv1,ENCDO719HIY
1,22Rv1_treated_with_10_nM_17β-hydroxy-5α-andros...,prostate,adult,"22Rv1, treated with 10 nM 17β-hydroxy-5α-andro...",ENCDO719HIY
2,8988T_ENCDO000ACP,pancreas,adult,8988T,ENCDO000ACP
3,A172_ENCDO934VEN,brain,adult,A172,ENCDO934VEN
4,A549_ENCDO000AAZ,lung,adult,A549,ENCDO000AAZ
...,...,...,...,...,...
1883,with_squamous_cell_carcinoma_skin_epidermis_ti...,skin,adult,"skin epidermis, male adult (75 years) with squ...",ENCDO317WFZ
1884,with_squamous_cell_carcinoma_skin_epidermis_ti...,skin,adult,"skin epidermis, male adult (78 years) with squ...",ENCDO054KII
1885,with_squamous_cell_carcinoma_skin_epidermis_ti...,skin,adult,"skin epidermis, male adult (84 years) with squ...",ENCDO799UNP
1886,WTC11_ENCDO882UJI,skin,adult,WTC11,ENCDO882UJI


# gencode parse

In [1]:
from gtfparse import read_gtf
import pandas as pd

In [ ]:
gencode_raw = "/panfs/accrepfs.vampire/data/ruderferlab/projects/ctcf/resources/data/gencode/v47/gencode.v47.basic.annotation.gtf"
gencode = read_gtf(gencode_raw).to_pandas()

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'transcript_support_level', 'ont', 'havana_transcript', 'protein_id', 'ccdsid', 'artif_dupl']


In [7]:
gencode.head()

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,...,exon_number,exon_id,hgnc_id,havana_gene,transcript_support_level,ont,havana_transcript,protein_id,ccdsid,artif_dupl
0,chr1,HAVANA,gene,11121,24894,NaN,+,0,ENSG00000290825.2,lncRNA,...,,,,,,,,,,
1,chr1,HAVANA,transcript,11426,14409,NaN,+,0,ENSG00000290825.2,lncRNA,...,,,,,,,,,,
2,chr1,HAVANA,exon,11426,11671,NaN,+,0,ENSG00000290825.2,lncRNA,...,1,ENSE00004248702.1,,,,,,,,
3,chr1,HAVANA,exon,12010,12227,NaN,+,0,ENSG00000290825.2,lncRNA,...,2,ENSE00004248735.1,,,,,,,,
4,chr1,HAVANA,exon,12613,12721,NaN,+,0,ENSG00000290825.2,lncRNA,...,3,ENSE00003582793.1,,,,,,,,


In [14]:
genecode_tx = gencode[(gencode['feature'] == 'transcript') & (gencode['gene_type'] == 'protein_coding')]

In [21]:
genecode_sub = genecode_tx[["transcript_id",  "gene_id", "gene_name", "gene_type", "seqname", "start", "end"]]

In [24]:
genecode_sub['length'] = genecode_sub['end'] - genecode_sub['start']

/tmp/ipykernel_231787/217293794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genecode_sub['length'] = genecode_sub['end'] - genecode_sub['start']


In [25]:
genecode_sub.head(10)

,transcript_id,gene_id,gene_name,gene_type,seqname,start,end,length
164,ENST00000641515.2,ENSG00000186092.7,OR4F5,protein_coding,chr1,65419,71585,6166
584,ENST00000426406.4,ENSG00000284733.2,OR4F29,protein_coding,chr1,450740,451678,938
816,ENST00000332831.5,ENSG00000284662.2,OR4F16,protein_coding,chr1,685716,686654,938
1115,ENST00000616016.5,ENSG00000187634.13,SAMD11,protein_coding,chr1,923923,944574,20651
1148,ENST00000618323.5,ENSG00000187634.13,SAMD11,protein_coding,chr1,923923,944574,20651
1181,ENST00000342066.8,ENSG00000187634.13,SAMD11,protein_coding,chr1,925731,944574,18843
1215,ENST00000327044.7,ENSG00000188976.11,NOC2L,protein_coding,chr1,944203,959256,15053
1259,ENST00000338591.8,ENSG00000187961.15,KLHL17,protein_coding,chr1,960584,965719,5135
1289,ENST00000379410.8,ENSG00000187583.11,PLEKHN1,protein_coding,chr1,966482,975865,9383
1326,ENST00000379407.7,ENSG00000187583.11,PLEKHN1,protein_coding,chr1,966502,975008,8506


In [26]:
 genecode_sub.sort_values(by=["gene_id", "length"], ascending=[True, False])

,transcript_id,gene_id,gene_name,gene_type,seqname,start,end,length
2183268,ENST00000612152.4,ENSG00000000003.16,TSPAN6,protein_coding,chrX,100627109,100637104,9995
2183247,ENST00000373020.9,ENSG00000000003.16,TSPAN6,protein_coding,chrX,100627108,100636806,9698
2183227,ENST00000373031.5,ENSG00000000005.6,TNMD,protein_coding,chrX,100584936,100599885,14949
2046719,ENST00000683466.1,ENSG00000000419.14,DPM1,protein_coding,chr20,50935020,50959140,24120
2046646,ENST00000371582.8,ENSG00000000419.14,DPM1,protein_coding,chr20,50934867,50958555,23688
...,...,...,...,...,...,...,...,...
666650,ENST00000509903.5,ENSG00000310517.1,CAST,protein_coding,chr5,96702806,96771679,68873
667153,ENST00000325674.11,ENSG00000310517.1,CAST,protein_coding,chr5,96743627,96774679,31052
667073,ENST00000508579.5,ENSG00000310517.1,CAST,protein_coding,chr5,96741874,96772813,30939
667114,ENST00000515663.5,ENSG00000310517.1,CAST,protein_coding,chr5,96743540,96772706,29166


In [27]:
# select longest transcript for each gene
genecode_sub = genecode_sub.sort_values(by=["gene_id", "length"], ascending=[True, False])
genecode_sub = genecode_sub.drop_duplicates(subset=["gene_id"], keep="first")

In [ ]:
# sort by chrom, start, end
genecode_sub = genecode_sub.sort_values(by=["seqname", "start", "end"])

In [36]:
genecode_sub

,transcript_id,gene_id,gene_name,gene_type,seqname,start,end,length
164,ENST00000641515.2,ENSG00000186092.7,OR4F5,protein_coding,chr1,65419,71585,6166
584,ENST00000426406.4,ENSG00000284733.2,OR4F29,protein_coding,chr1,450740,451678,938
816,ENST00000332831.5,ENSG00000284662.2,OR4F16,protein_coding,chr1,685716,686654,938
1115,ENST00000616016.5,ENSG00000187634.13,SAMD11,protein_coding,chr1,923923,944574,20651
1215,ENST00000327044.7,ENSG00000188976.11,NOC2L,protein_coding,chr1,944203,959256,15053
...,...,...,...,...,...,...,...,...
2225533,ENST00000361335.1,ENSG00000212907.2,MT-ND4L,protein_coding,chrM,10470,10766,296
2225540,ENST00000361381.2,ENSG00000198886.2,MT-ND4,protein_coding,chrM,10760,12137,1377
2225554,ENST00000361567.2,ENSG00000198786.2,MT-ND5,protein_coding,chrM,12337,14148,1811
2225560,ENST00000361681.2,ENSG00000198695.2,MT-ND6,protein_coding,chrM,14149,14673,524


In [38]:
genecode_sub[['gene_name','gene_id', 'transcript_id']].to_csv("data/gencode/v47/gencode.v47.basic.annotation.parsed.pc.longest_tx.tsv", sep="\t", index=False, header=False)


# activity

In [ ]:
"""rdhs-activity.py"""

import numpy as np
import pandas as pd

# Snakemake
CCRES = snakemake.input[0]  # type: ignore
SIGNAL_SUMMARY = snakemake.input[1]  # type: ignore
OUTPUT = snakemake.output[0]  # type: ignore


# ------------- #
# Functions     #
# ------------- #


def read_rdhslist(filepath: str) -> pd.DataFrame:
    """Returns CCRE bed file as DataFrame"""
    return pd.read_csv(
        filepath,
        sep="\t",
        header=None,
        engine="c",
        usecols=[3],
        names=["rDHS"],
        dtype={"rDHS": str},
    )["rDHS"].to_list()


def read_signal_summary(filepath: str) -> pd.DataFrame:
    dtype = {
        "rDHS": str,
        "sum_signal": float,
        "num_signal": int,
    }
    return pd.read_csv(
        filepath,
        sep="\t",
        dtype=dtype,
        engine="c",
    )


def main():
    """Main"""
    # Read inputs
    rdhs_list = read_rdhslist(CCRES)
    signal_summary = read_signal_summary(SIGNAL_SUMMARY)

    # Subset to rDHS to known ccre
    signal_summary = signal_summary[signal_summary["rDHS"].isin(rdhs_list)]

    # Calculate activity
    signal_summary["activity"] = signal_summary["sum_signal"] / np.sqrt(
        signal_summary["num_signal"]
    )

    # Add quantile across all rDHS
    signal_summary["quantile_rdhswide"] = pd.qcut(
        signal_summary["activity"], 100, labels=[i for i in range(1, 101)]
    )

    # Tidy up
    signal_summary = signal_summary[["rDHS", "activity", "quantile_rdhswide"]].round(4)
    dtypes = {"rDHS": str, "activity": float, "quantile_rdhswide": int}
    signal_summary = signal_summary.astype(dtypes)

    # Write output
    signal_summary.to_csv(OUTPUT, sep="\t", index=False)